In [ ]:
import numpy as np 
import pandas as pd 
import warnings
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
import itertools

from pylab import rcParams

warnings.filterwarnings('ignore')
rcParams['figure.figsize'] = 18, 15
sns.set_style('darkgrid')

In [ ]:
''' reading dataset '''
df_city_day = pd.read_csv('city_day.csv')

In [ ]:
''' displaying 5 rows '''
df_city_day.head()

In [ ]:
''' checking null values '''
df_city_day.isnull().sum()

In [ ]:
''' shape of dataset '''
df_city_day.shape

In [ ]:
''' percentage of missing values in each column '''
for c in df_city_day.columns:
    null_values = df_city_day[c].isna().sum()
    percentage = (null_values / len(df_city_day)) * 100
    print("In {}, mean of null value is: {}".format(c, percentage))
    print("-" * 100)

In [ ]:
''' filling null values with mean of each column '''
for c in df_city_day.columns:
    if df_city_day[c].isna().sum() > 0:
        if df_city_day[c].dtype == 'float64':
            df_city_day[c] = df_city_day[c].fillna(df_city_day[c].mean())
        elif df_city_day[c].dtype == 'object':
            df_city_day[c] = df_city_day[c].fillna(df_city_day[c].value_counts().index[0])

In [ ]:
'''again checking null values'''
df_city_day.isna().sum()

In [ ]:
''' info of dataset '''
df_city_day.info()

In [ ]:
''' count of cities '''
city_label = df_city_day.City.value_counts().nlargest(10)

''' barplot '''
plt.figure(figsize=(10, 5))
plt.xticks(rotation=75)
sns.barplot(city_label.index, city_label)
plt.xlabel('City', fontsize=20)
plt.ylabel('Count', fontsize=20);

In [ ]:
''' making date as index of data '''
# df.head
df_city_day.index = pd.DatetimeIndex(df_city_day['Date'])
df_city_day.head()

In [ ]:
''' dropping some columns '''
df_city_day.drop(['City' , 'PM2.5' , 'PM10','Benzene' , 'Toluene', 'Xylene'  ,'AQI' ,'AQI_Bucket'], axis=1, inplace=True)

In [ ]:
''' after dropping some columns, data looks like '''
df_city_day.head()

In [ ]:
# ''' Considering the pollutant NO  '''
# decomp_no = sm.tsa.seasonal_decompose(df_city_day['NO'].resample('2W').mean(), model='additive')
# fig1 = decomp_no.plot()
# plt.show()

decomp_no = sm.tsa.seasonal_decompose(df_city_day['NO'].resample('1W').mean(),model='additive')
plt.figure(figsize=(16,18))
fig1 = decomp_no.plot()
plt.show()

In [ ]:
# ''' SARIMA Model for NO '''

# a = b = c = range(0, 3)
# p = list(itertools.product(a, b, c))
# seas_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(a, b, c))]
# print('SARIMAX: {} x {}'.format(p[1], seasonal_pdq[1]))
# print('SARIMAX: {} x {}'.format(p[1], seasonal_pdq[2]))
# print('SARIMAX: {} x {}'.format(p[2], seasonal_pdq[3]))
# print('SARIMAX: {} x {}'.format(p[2], seasonal_pdq[4]))
a = b = c = range(0,3)
p = list(itertools.product(a,b,c))
seas_pdq = [(x[0],x[1],x[2],12) for x in list(itertools.product(a,b,c))]
print('SARIMAX : {} x {}'.format(p[1],seas_pdq[1]))
print('SARIMAX : {} x {}'.format(p[1],seas_pdq[2]))
print('SARIMAX : {} x {}'.format(p[2],seas_pdq[3]))
print('SARIMAX : {} x {}'.format(p[2],seas_pdq[4]))

In [ ]:
# for params in p:
#     for p_seasonal in seas_pdq:
#         try:
#             m = sm.tsa.statespace.SARIMAX(df_city_day['NO'].resample('2W').mean(),order=param,seasonal_order=p_seasonal, 
#                                           enforce_stationarity=False, enforce_invertibility=False)
#             results = m.fit()
#         except:
#             continue

for params in p:
    for p_seasonal in seas_pdq:
        try :
            m = sm.tsa.statespace.SARIMAX(df_city_day['NO'].resample('2W').mean(),order = params,seasonal_order = p_seasonal,
                                         enforce_stationarity = False,enforce_invertibility = False)
            results = m.fit()
        except:
            continue

In [ ]:
''' model '''
model = sm.tsa.statespace.SARIMAX(df_city_day['NO'].resample('2W').mean(), order=(2, 1, 1), seasonal_order=(1, 1, 0, 12),
                                  enforce_stationarity=False, enforce_invertibility=False)

res = model.fit()
print(res.summary().tables[1])

In [ ]:
''' plotting '''
res.plot_diagnostics(figsize=(16, 8))
plt.show()

In [ ]:
y_pred = res.get_prediction(start=pd.to_datetime('2017-01-01'), dynamic=False)
y_pred_ci = y_pred.conf_int()

ax = df_city_day['NO'].resample('2W').mean()['2017':].plot(label='observed')
y_pred.predicted_mean.plot(ax=ax, label='Forecast', alpha=.7, figsize=(14, 7))

ax.fill_between(y_pred_ci.index,
                y_pred_ci.iloc[:, 0],
                y_pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('NO Concentration')

plt.legend(['observed', 'Forecast'])
plt.show()

In [ ]:
''' calculating mse and rmse '''

y_f = y_pred.predicted_mean
actual = df_city_day['NO'].resample('2W').mean()['2017-01-01':]
mse = ((y_f - actual) ** 2).mean()

print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

In [ ]:
''' considering the pollutant SO2  '''

df_city_day['NO'].resample('2W').mean().plot(figsize=(15, 6))
plt.show()

In [ ]:
decomp_so2 = sm.tsa.seasonal_decompose(df_city_day['NO'].resample('2W').mean(), model='additive')
fig_so2 = decomp_so2.plot()
plt.show()

In [ ]:
''' SARIMA for SO2 '''

a = b = c = range(0, 3)
p = list(itertools.product(a, b, c))
seas_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(a, b, c))]
print('SARIMAX: {} x {}'.format(p[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(p[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(p[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(p[2], seasonal_pdq[4]))

In [ ]:
for params in p:
    for p_seasonal in seas_pdq:
        try:
            m = sm.tsa.statespace.SARIMAX(df_city_day['SO2'].resample('2W').mean(),order=param,seasonal_order=p_seasonal, 
                                          enforce_stationarity=False, enforce_invertibility=False)
            results = m.fit()
        except:
            continue

In [ ]:
mod = sm.tsa.statespace.SARIMAX(df_city_day['SO2'].resample('2W').mean(), order=(2, 1, 1), seasonal_order=(1, 1, 0, 12),
                                enforce_stationarity=False, enforce_invertibility=False)
res = mod.fit()
print(results.summary().tables[1])

In [ ]:
''' plotting '''
res.plot_diagnostics(figsize=(16, 8))
plt.show()

In [ ]:
y_pred = res.get_prediction(start=pd.to_datetime('2017-01-01'), dynamic=False)
y_pred_ci = y_pred.conf_int()

ax = df_city_day['SO2'].resample('2W').mean()['2017':].plot(label='observed')
y_pred.predicted_mean.plot(ax=ax, label='Forecast', alpha=.7, figsize=(14, 7))

ax.fill_between(y_pred_ci.index,
                y_pred_ci.iloc[:, 0],
                y_pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('NO Concentration')

plt.legend(['observed', 'Forecast'])
plt.show()

In [ ]:
''' calculating mse and rmse '''

y_f = y_pred.predicted_mean
actual = df_city_day['SO2'].resample('2W').mean()['2017-01-01':]
mse = ((y_f - actual) ** 2).mean()

print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))